# Table of Contents
## 1. Import Data and Checks
## 2. Create Customer Loyalty Flag
## 3. Create Flag for Price Ranges
## 4. Create Flag for High and Low Spenders
## 5. Export Data

# 1. Import Data and Checks

In [1]:
# import libraries
import pandas as pd
import numpy as np
import os

In [2]:
# create path
path = r'C:\Users\18602\Documents\Data Analytics\Data Immersion\Month 4\Instacart Basket Analysis'

In [3]:
# import data
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_large.pkl'))

In [4]:
# rename for easy of coding
df = ords_prods_merge

In [5]:
# check shape
df.shape

(32641279, 20)

In [6]:
#find column names
df.columns

Index(['Unnamed: 0_x', 'order_id', 'user_id', 'order_number',
       'order_day_of_week', 'order_hour_of_day', 'days_since_prior_order',
       'product_id', 'add_to_cart_order', 'reordered', '_merge',
       'Unnamed: 0_y', 'product_name', 'aisle_id', 'department_id', 'prices',
       'price_range_loc', 'busiest_day', 'busiest_days',
       'busiest_period_of_day'],
      dtype='object')

In [7]:
# chech output
df.head()

,Unnamed: 0_x,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,Unnamed: 0_y,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_days,busiest_period_of_day
0,0.0,2539329.0,1.0,1.0,2.0,8.0,NaN,196.0,1.0,0.0,both,195.0,Soda,77.0,7.0,9.0,Mid=range product,Regularly busy,Regularly busy,Average Orders
1,1.0,2398795.0,1.0,2.0,3.0,7.0,15.0,196.0,1.0,1.0,both,195.0,Soda,77.0,7.0,9.0,Mid=range product,Regularly busy,Least busy days,Average Orders
2,2.0,473747.0,1.0,3.0,3.0,12.0,21.0,196.0,1.0,1.0,both,195.0,Soda,77.0,7.0,9.0,Mid=range product,Regularly busy,Least busy days,Most Orders
3,3.0,2254736.0,1.0,4.0,4.0,7.0,29.0,196.0,1.0,1.0,both,195.0,Soda,77.0,7.0,9.0,Mid=range product,Least busy,Least busy days,Average Orders
4,4.0,431534.0,1.0,5.0,4.0,15.0,28.0,196.0,1.0,1.0,both,195.0,Soda,77.0,7.0,9.0,Mid=range product,Least busy,Least busy days,Most Orders


In [8]:
# drop unnecessary column busiest day and unnamed columns

df = df.drop(columns=['busiest_day', 'Unnamed: 0_x', 'Unnamed: 0_y'])

# 2. Create Customer Loyalty Flag

In [9]:
# create average orders of department_id
df.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1.0,15.457838
2.0,17.277920
3.0,17.170395
4.0,17.811403
5.0,15.215751
6.0,16.439806
7.0,17.225802
8.0,15.340650


The average number of orders per user is lower than in the partial data set in the visible data output

In [10]:
# create max_order column with groupby for loyalty flag

df['max_order'] = df.groupby(['user_id'])['order_number'].transform(np.max)

In [11]:
# create loyalty flag parameters
df.loc[df['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
df.loc[(df['max_order'] <= 40) & (df['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
df.loc[df['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [12]:
# check loyalty counts
df['loyalty_flag'].value_counts(dropna = False)

Regular customer    16381472
Loyal customer      10679578
New customer         5580218
NaN                       11
Name: loyalty_flag, dtype: int64

In [13]:
# check output of loyalty flag
df[['user_id', 'loyalty_flag', 'order_number', 'max_order']].head(20)

,user_id,loyalty_flag,order_number,max_order
0,1.0,Regular customer,1.0,11.0
1,1.0,Regular customer,2.0,11.0
2,1.0,Regular customer,3.0,11.0
3,1.0,Regular customer,4.0,11.0
4,1.0,Regular customer,5.0,11.0
5,1.0,Regular customer,6.0,11.0
6,1.0,Regular customer,7.0,11.0
7,1.0,Regular customer,8.0,11.0
8,1.0,Regular customer,9.0,11.0
9,1.0,Regular customer,10.0,11.0


In [14]:
# check average price of types of customer
df.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.377144  1.0  99999.0
New customer      13.208437  1.0  99999.0
Regular customer  12.616225  1.0  99999.0

#The average prices seem to be slightly higher in New Customers than in Regular and Loyal customers. 
The min and max don't entirely make sense. No is priced is 99999 so I'm going to assume that my data needs to be cleaned or that the min and max are not aggregating price correctly.

# 3. Create Flag for Price Ranges

In [15]:
# create df to check prices
df2 = df['prices']

In [16]:
# check for descriptive statistics of price
df2.describe()

count    3.240487e+07
mean     1.198022e+01
std      4.956554e+02
min      1.000000e+00
25%      4.200000e+00
50%      7.400000e+00
75%      1.130000e+01
max      9.999900e+04
Name: prices, dtype: float64

Based on this analysis I'm going to conclude some of the pricing is innacurate. Not sure how to drop or exclude those specific rows but we are going to assume it is skewing our results a little high but doing so evenly to all three categories.

In [17]:
# eliminate maximum rows
pd.options.display.max_rows = None

In [40]:
#check value counts

df2.value_counts()

12.3       624261
10.3       510496
4.0        447272
6.3        438467
1.3        425691
7.3        411924
8.2        404015
1.4        392209
6.7        379126
4.5        367223
4.4        357026
6.8        354169
7.9        342988
14.3       334812
13.2       333183
5.7        323997
1.8        321185
9.1        320783
4.9        318220
8.8        318091
3.8        315426
6.9        314196
14.0       311640
12.8       309727
11.4       303015
3.6        300127
1.6        299352
2.0        286259
11.8       284476
5.3        284243
7.0        282884
14.7       279291
6.5        278357
8.3        276230
4.1        268269
2.9        264700
1.2        256258
8.6        254009
3.2        253782
5.2        251509
5.5        250136
3.1        249256
2.5        246421
4.2        245755
6.1        244209
1.1        242782
2.3        242051
5.8        241047
7.5        239261
14.1       238211
3.3        238041
10.5       237657
7.4        233506
9.0        228576
8.4        227493
5.0       

So this is interesting. Based on this analysis I can see we have two price points that are clearly incorrect. First is the 14900.0, which based on some context looks like it was supposed to be 14.9.

The other one is that the 99999.0 has 698 results, which is still relatively small but not insignificant (.3% of the data).

The solution would be to transform the 14900.0 and create a data frame without the 99999 average price. If we wanted to potentially use those products but couldn't find the correct information, I'd recommend inserting a mean into that slot that was based on a data frame that took out the 99999. This would lead to a lower average price point. However, for the purposes of this exercise I will leave that for another day.

In [15]:
#attempted to filter out all rows. Got error message.

df_filtered = (df.loc[df['prices'] < 14899])

I have decided to keep coding since my calculations will still work the same. If I get updated numbers I can use the same code to accomplish the task.

In [16]:
# make column for average price by user
df['average_order'] = df.groupby(['user_id'])['prices'].transform(np.mean)

In [17]:
# check for new column
df.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,price_range_loc,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_order
0,2539329.0,1.0,1.0,2.0,8.0,NaN,196.0,1.0,0.0,both,Soda,77.0,7.0,9.0,Mid=range product,Regularly busy,Average Orders,11.0,Regular customer,6.367797
1,2398795.0,1.0,2.0,3.0,7.0,15.0,196.0,1.0,1.0,both,Soda,77.0,7.0,9.0,Mid=range product,Least busy days,Average Orders,11.0,Regular customer,6.367797
2,473747.0,1.0,3.0,3.0,12.0,21.0,196.0,1.0,1.0,both,Soda,77.0,7.0,9.0,Mid=range product,Least busy days,Most Orders,11.0,Regular customer,6.367797
3,2254736.0,1.0,4.0,4.0,7.0,29.0,196.0,1.0,1.0,both,Soda,77.0,7.0,9.0,Mid=range product,Least busy days,Average Orders,11.0,Regular customer,6.367797
4,431534.0,1.0,5.0,4.0,15.0,28.0,196.0,1.0,1.0,both,Soda,77.0,7.0,9.0,Mid=range product,Least busy days,Most Orders,11.0,Regular customer,6.367797


# 4. Create Flag for High and Low Spenders

In [18]:
# assign flags for low and high spenders
df.loc[df['average_order'] >9.99, 'spender'] = 'High Spender'
df.loc[df['average_order'] <9.99, 'spender'] = 'Low Spender'

In [30]:
# check output 

df.head(100)

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,...,department_id,prices,price_range_loc,busiest_days,busiest_period_of_day,max_order,loyalty_flag,averge_order,average_order,spender
0,2539329.0,1.0,1.0,2.0,8.0,NaN,196.0,1.0,0.0,both,...,7.0,9.0,Mid=range product,Regularly busy,Average Orders,6.367797,Regular customer,6.367797,6.367797,Low Spender
1,2398795.0,1.0,2.0,3.0,7.0,15.0,196.0,1.0,1.0,both,...,7.0,9.0,Mid=range product,Least busy days,Average Orders,6.367797,Regular customer,6.367797,6.367797,Low Spender
2,473747.0,1.0,3.0,3.0,12.0,21.0,196.0,1.0,1.0,both,...,7.0,9.0,Mid=range product,Least busy days,Most Orders,6.367797,Regular customer,6.367797,6.367797,Low Spender
3,2254736.0,1.0,4.0,4.0,7.0,29.0,196.0,1.0,1.0,both,...,7.0,9.0,Mid=range product,Least busy days,Average Orders,6.367797,Regular customer,6.367797,6.367797,Low Spender
4,431534.0,1.0,5.0,4.0,15.0,28.0,196.0,1.0,1.0,both,...,7.0,9.0,Mid=range product,Least busy days,Most Orders,6.367797,Regular customer,6.367797,6.367797,Low Spender
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3226575.0,360.0,1.0,5.0,12.0,NaN,196.0,1.0,0.0,both,...,7.0,9.0,Mid=range product,Regularly busy,Most Orders,10.006250,New customer,10.006250,10.006250,High Spender
96,1469869.0,377.0,3.0,5.0,17.0,3.0,196.0,9.0,0.0,both,...,7.0,9.0,Mid=range product,Regularly busy,Average Orders,8.496552,New customer,8.496552,8.496552,Low Spender
97,1927023.0,387.0,2.0,4.0,10.0,22.0,196.0,3.0,0.0,both,...,7.0,9.0,Mid=range product,Least busy days,Most Orders,7.396610,New customer,7.396610,7.396610,Low Spender
98,858092.0,420.0,4.0,1.0,19.0,30.0,196.0,2.0,0.0,both,...,7.0,9.0,Mid=range product,Busiest days,Average Orders,7.387805,Regular customer,7.387805,7.387805,Low Spender


In [52]:
# calculate the median for days_since_prior_order
df['median_frequency'] = df.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [55]:
# create flag for days_since_prior_order for frequency
df.loc[df['median_frequency'] > 19, 'frequency'] = 'Non Frequent'

In [57]:
df.loc[(df['median_frequency'] <= 19) & (df['median_frequency'] > 10), 'frequency'] = 'Regular Customer'

In [58]:
df.loc[df['median_frequency'] <= 9, 'frequency'] = 'Frequent Customer'

In [59]:
df.columns

Index(['order_id', 'user_id', 'order_number', 'order_day_of_week',
       'order_hour_of_day', 'days_since_prior_order', 'product_id',
       'add_to_cart_order', 'reordered', '_merge', 'product_name', 'aisle_id',
       'department_id', 'prices', 'price_range_loc', 'busiest_days',
       'busiest_period_of_day', 'max_order', 'loyalty_flag', 'averge_order',
       'average_order', 'spender', 'frequency', 'median_frequency'],
      dtype='object')

In [60]:
df.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,...,price_range_loc,busiest_days,busiest_period_of_day,max_order,loyalty_flag,averge_order,average_order,spender,frequency,median_frequency
0,2539329.0,1.0,1.0,2.0,8.0,NaN,196.0,1.0,0.0,both,...,Mid=range product,Regularly busy,Average Orders,6.367797,Regular customer,6.367797,6.367797,Low Spender,Non Frequent,20.0
1,2398795.0,1.0,2.0,3.0,7.0,15.0,196.0,1.0,1.0,both,...,Mid=range product,Least busy days,Average Orders,6.367797,Regular customer,6.367797,6.367797,Low Spender,Non Frequent,20.0
2,473747.0,1.0,3.0,3.0,12.0,21.0,196.0,1.0,1.0,both,...,Mid=range product,Least busy days,Most Orders,6.367797,Regular customer,6.367797,6.367797,Low Spender,Non Frequent,20.0
3,2254736.0,1.0,4.0,4.0,7.0,29.0,196.0,1.0,1.0,both,...,Mid=range product,Least busy days,Average Orders,6.367797,Regular customer,6.367797,6.367797,Low Spender,Non Frequent,20.0
4,431534.0,1.0,5.0,4.0,15.0,28.0,196.0,1.0,1.0,both,...,Mid=range product,Least busy days,Most Orders,6.367797,Regular customer,6.367797,6.367797,Low Spender,Non Frequent,20.0


I'm not sure how I got two average order columns. There was a lot going on in this code. Also I'm curious as to why my markdowns have two different foints going on. Some look more like coding and some look more like the output, but I'm not selecting anything differently consciously.

# 5. Export Data

In [21]:
#export to pickle

df.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_large_4_8.pkl'))

In [19]:
df['loyalty_flag'].value_counts()

Regular customer    16381472
Loyal customer      10679578
New customer         5580218
Name: loyalty_flag, dtype: int64

In [20]:
df.shape

(32641279, 21)